In [1]:
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster

In [2]:
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("Champ de Mars, Paris, France")

In [3]:
print(location.address)
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

Champ de Mars, Rue Savorgnan de Brazza, Quartier du Gros-Caillou, Paris, Île-de-France, France métropolitaine, 75007, France
Latitude = 48.85614465, Longitude = 2.297820393322227


In [4]:
df = pd.read_csv("dropoff_locations.csv")
df.head()

,Number,St,Unit,City,State,Delivery_Rt_ID
0,350,3rd St,#1211,Cambridge,MA,1
1,164,5th St,#1,Cambridge,MA,1
2,217,Auburn St,#1,Cambridge,MA,1
3,54,Berkshire St,#1,Cambridge,MA,1
4,243,Broadway,#610,Cambridge,MA,1


In [5]:
df['ADDRESS'] = df['Number'].astype(str) + ' ' + \
                df['St'] + ',' + \
                df['City'] + ',' + \
                df['State'] + ',' + ' USA'   

df.head()

,Number,St,Unit,City,State,Delivery_Rt_ID,ADDRESS
0,350,3rd St,#1211,Cambridge,MA,1,"350 3rd St,Cambridge,MA, USA"
1,164,5th St,#1,Cambridge,MA,1,"164 5th St,Cambridge,MA, USA"
2,217,Auburn St,#1,Cambridge,MA,1,"217 Auburn St,Cambridge,MA, USA"
3,54,Berkshire St,#1,Cambridge,MA,1,"54 Berkshire St,Cambridge,MA, USA"
4,243,Broadway,#610,Cambridge,MA,1,"243 Broadway,Cambridge,MA, USA"


In [6]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(locator.geocode, min_delay_seconds=1.3)
df['location'] = df['ADDRESS'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('6 Clifton Place,Cambridge,MA, USA',), **{}).
Traceback (most recent call last):
  File "/Users/ericenglin/opt/anaconda3/lib/python3.7/site-packages/geopy/geocoders/base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "/Users/ericenglin/opt/anaconda3/lib/python3.7/urllib/request.py", line 525, in open
    response = self._open(req, data)
  File "/Users/ericenglin/opt/anaconda3/lib/python3.7/urllib/request.py", line 543, in _open
    '_open', req)
  File "/Users/ericenglin/opt/anaconda3/lib/python3.7/urllib/request.py", line 503, in _call_chain
    result = func(*args)
  File "/Users/ericenglin/opt/anaconda3/lib/python3.7/urllib/request.py", line 1360, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "/Users/ericenglin/opt/anaconda3/lib/python3.7/urllib/request.py", line 1320, in do_open
    r = h.getresponse()
  File "/Users/ericenglin/opt/a

In [7]:
# split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
df['point_location'] = df["longitude"].astype(str) + "," + df["latitude"].astype(str)
df.head()

,Number,St,Unit,City,State,Delivery_Rt_ID,ADDRESS,location,point,latitude,longitude,altitude,point_location
0,350,3rd St,#1211,Cambridge,MA,1,"350 3rd St,Cambridge,MA, USA","(350, Third Street, East Cambridge, Cambridge,...","(42.363930822916664, -71.0835468125, 0.0)",42.363931,-71.083547,0.0,"-71.0835468125,42.363930822916664"
1,164,5th St,#1,Cambridge,MA,1,"164 5th St,Cambridge,MA, USA","(162;164, Fifth Street, East Cambridge, Cambri...","(42.36820405, -71.08375762804951, 0.0)",42.368204,-71.083758,0.0,"-71.08375762804951,42.36820405"
2,217,Auburn St,#1,Cambridge,MA,1,"217 Auburn St,Cambridge,MA, USA","(217, Auburn Street, Cambridgeport, Cambridge,...","(42.36471025, -71.10675534090909, 0.0)",42.364710,-71.106755,0.0,"-71.10675534090909,42.36471025"
3,54,Berkshire St,#1,Cambridge,MA,1,"54 Berkshire St,Cambridge,MA, USA","(54;56, Berkshire Street, East Cambridge, Camb...","(42.3696425, -71.09104734310242, 0.0)",42.369642,-71.091047,0.0,"-71.09104734310242,42.3696425"
4,243,Broadway,#610,Cambridge,MA,1,"243 Broadway,Cambridge,MA, USA","(243, Broadway, Inman Square, Cambridge, Middl...","(42.367239, -71.09552, 0.0)",42.367239,-71.095520,0.0,"-71.09552,42.367239"


In [8]:
df.loc[df['point'].isnull()]

,Number,St,Unit,City,State,Delivery_Rt_ID,ADDRESS,location,point,latitude,longitude,altitude,point_location
66,1000,Jackon Pl,#37,Cambridge,MA,7,"1000 Jackon Pl,Cambridge,MA, USA",None,None,NaN,NaN,NaN,"nan,nan"
67,117,Jackon Pl,NaN,Cambridge,MA,7,"117 Jackon Pl,Cambridge,MA, USA",None,None,NaN,NaN,NaN,"nan,nan"


In [9]:
df.to_csv("dropoff_locations_clean.csv")

In [10]:
df_new = df.dropna(subset=['point'], how='all')
df_new.latitude.isnull().sum()

0

In [11]:
len(df_new), len(df)

(343, 345)

In [12]:
map1 = folium.Map(
    location=[42.362750, -71.101182],
    tiles='cartodbpositron',
    zoom_start=12,
)

In [13]:
df_new.apply(lambda row:folium.Marker(
    location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1

In [14]:
locations = df_new[['latitude', 'longitude']]
locationlist = locations.values.tolist()

In [16]:
import hdbscan
import numpy as np
rads = np.radians(locationlist)
clusterer = hdbscan.HDBSCAN(min_cluster_size=10, min_samples=1, metric='haversine')
cluster_labels = clusterer.fit_predict(locationlist)
df_new['hbdscan_cluster']=cluster_labels

np.unique(cluster_labels)

/Users/ericenglin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [17]:
df_new.hbdscan_cluster.value_counts()

 8     38
 1     36
 5     29
 4     29
 9     28
-1     28
 6     27
 10    24
 12    23
 11    23
 2     17
 3     16
 7     15
 0     10
Name: hbdscan_cluster, dtype: int64

In [18]:
len(np.unique(cluster_labels))

14

In [124]:
color_dict = {-1:'darkgreen',0:'darkpurple',1:'green',2:'purple',3:'red',4:'gray',5:'lightblue',
             6:'beige',7:'cadetblue',8:'black',9:'pink',10:'darkred',11:'lightred',
             12:'darkblue',13:'lightgreen',13:'white',14:'darkgreen',15:'darkpurple',16:'green',
              17:'purple',18:'red',19:'gray',20:'lightblue',
             21:'beige',22:'cadetblue',23:'black',24:'pink',25:'darkred',26:'lightred',
             27:'darkblue',28:'lightgreen',29:'white',30:'gray'}

In [125]:
df_new['hbdscan_color']=df_new['hbdscan_cluster'].map(color_dict)

In [25]:
df_new=df_new.reset_index()

In [29]:
map = folium.Map(location=[42.379750, -71.101182], zoom_start=13)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=('Driver Group: '+str(df_new['hbdscan_cluster'][point])),
                 icon=folium.Icon(color=df_new["color"][point])).add_to(map)
    
map.save('DeliveryMaps.html')
map

'350, Third Street, East Cambridge, Cambridge, Middlesex County, Massachusetts, 02142, United States of America'

In [37]:
def find_nth(string, substring, n):
    if (n == 1):
        return string.find(substring)
    else:
        return string.find(substring, find_nth(string, substring, n - 1) + 1)
    

In [93]:
neighborhood_list = ['Old Cambridge','Strawberry Hill','Inman Square',
                    'Cambridgeport','East Cambridge','Huron Village',
                    'North Cambridge','Porter Square','Waverley','Avon Hill',
                    'Union Square','University Park']

In [89]:
for neighborhood in neighborhood_list:
    print(neighborhood)

Old Cambridge
Strawberry Hill
Inman Square
Cambridgeport
East Cambridge
Huron Village
North Cambridge
Porter Square
Waverley
Avon Hill
Union Square
University Park


In [ ]:
for x in 

In [96]:
neighborhood_list_temp = []

for x in df_new['location']:
    z=0
    for y in neighborhood_list:
        #print(y)
        if y in str(x):
            neighborhood_list_temp.append(y)
            z=1
    if z==0:
        neighborhood_list_temp.append('Cambridge')

In [97]:
df_new['neighborhood']=neighborhood_list_temp

In [98]:
df_new['neighborhood'].value_counts()

Cambridgeport      116
East Cambridge      69
Inman Square        44
Cambridge           40
North Cambridge     34
Strawberry Hill     12
Avon Hill            6
Huron Village        6
Porter Square        5
University Park      4
Old Cambridge        3
Waverley             3
Union Square         1
Name: neighborhood, dtype: int64

In [99]:
# Get one hot encoding of columns B
one_hot = pd.get_dummies(df_new['neighborhood'])
# Drop column B as it is now encoded
df_new = df_new.drop('neighborhood',axis = 1)
# Join the encoded df
df_new = df_new.join(one_hot)

In [101]:
df_new.columns

Index(['index', 'Number', 'St', 'Unit', 'City', 'State', 'Delivery_Rt_ID',
       'ADDRESS', 'location', 'point', 'latitude', 'longitude', 'altitude',
       'point_location', 'hbdscan_cluster', 'color', 'Avon Hill', 'Cambridge',
       'Cambridgeport', 'East Cambridge', 'Huron Village', 'Inman Square',
       'North Cambridge', 'Old Cambridge', 'Porter Square', 'Strawberry Hill',
       'Union Square', 'University Park', 'Waverley'],
      dtype='object')

In [106]:
df_temp = df_new.drop(columns = ['index', 'Number', 'St', 'Unit', 'City', 'State', 'Delivery_Rt_ID',
       'ADDRESS', 'location', 'point', 'altitude','point_location', 'hbdscan_cluster', 'color'])


In [139]:
!pip install k-means-constrained

     |████████████████████████████████| 163 kB 562 kB/s eta 0:00:01
     |████████████████████████████████| 24.0 MB 6.6 MB/s eta 0:00:01


In [145]:
from k_means_constrained import KMeansConstrained
# Convert DataFrame to matrix
mat = df_temp.values
# Using sklearn
km = KMeansConstrained(
     n_clusters=30,
     size_min=9,
     size_max=14,
     random_state=0
).fit(mat)
# Get cluster assignment labels
labels = km.labels_
# Format results as a DataFrame
df_new['kmeans_cluster'] = labels

In [146]:
df_new['kmeans_cluster'].value_counts()

29    14
20    14
1     14
3     14
12    14
18    14
19    14
14    14
24    14
21    13
17    13
15    13
28    13
27    12
6     12
23    11
22    10
2     10
4     10
0     10
9      9
8      9
7      9
11     9
5      9
25     9
26     9
13     9
16     9
10     9
Name: kmeans_cluster, dtype: int64

In [147]:
df_new['kmeans_color']=df_new['kmeans_cluster'].map(color_dict)

In [148]:
map = folium.Map(location=[42.379750, -71.101182], zoom_start=13)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=('Driver Group: '+str(df_new['kmeans_cluster'][point])),
                 icon=folium.Icon(color=df_new["kmeans_color"][point])).add_to(map)
    
map.save('DeliveryMaps_kmeans.html')
map

In [162]:
df_final = df.merge(df_new, on=['Number', 'St', 'Unit', 'City', 'State', 'Delivery_Rt_ID','ADDRESS',
                               'point','latitude','longitude','altitude','point_location'],
                    how='left')

In [163]:
df_final.columns

Index(['Number', 'St', 'Unit', 'City', 'State', 'Delivery_Rt_ID', 'ADDRESS',
       'location_x', 'point', 'latitude', 'longitude', 'altitude',
       'point_location', 'index', 'location_y', 'hbdscan_cluster', 'color',
       'Avon Hill', 'Cambridge', 'Cambridgeport', 'East Cambridge',
       'Huron Village', 'Inman Square', 'North Cambridge', 'Old Cambridge',
       'Porter Square', 'Strawberry Hill', 'Union Square', 'University Park',
       'Waverley', 'hbdscan_color', 'kmeans_clust', 'kmeans_cluster',
       'kmeans_color'],
      dtype='object')

In [164]:
df_final['Driver_ID']=df_final['hbdscan_cluster']
df_final['Driver_ID2']=df_final['kmeans_cluster']
df_final = df_final.drop(columns = ['index','altitude',
       'point_location', 'Avon Hill', 'Cambridge', 'Cambridgeport', 'East Cambridge',
       'Huron Village', 'Inman Square', 'North Cambridge', 'Old Cambridge',
       'Porter Square', 'Strawberry Hill', 'Union Square', 'University Park',
       'Waverley', 'location_x','location_y',
       'hbdscan_cluster', 'hbdscan_color'])

df_final.loc[df_final['Driver_ID'].isnull(),'Driver_ID']=99 #if no lat/long found, make driver id 99
df_final.Driver_ID = df_final.Driver_ID.astype(int)

df_final.head()

,Number,St,Unit,City,State,Delivery_Rt_ID,ADDRESS,point,latitude,longitude,color,kmeans_clust,kmeans_cluster,kmeans_color,Driver_ID,Driver_ID2
0,350,3rd St,#1211,Cambridge,MA,1,"350 3rd St,Cambridge,MA, USA","(42.363930822916664, -71.0835468125, 0.0)",42.363931,-71.083547,beige,27.0,19.0,gray,6,19.0
1,164,5th St,#1,Cambridge,MA,1,"164 5th St,Cambridge,MA, USA","(42.36820405, -71.08375762804951, 0.0)",42.368204,-71.083758,pink,27.0,19.0,gray,9,19.0
2,217,Auburn St,#1,Cambridge,MA,1,"217 Auburn St,Cambridge,MA, USA","(42.36471025, -71.10675534090909, 0.0)",42.364710,-71.106755,lightred,0.0,2.0,purple,11,2.0
3,54,Berkshire St,#1,Cambridge,MA,1,"54 Berkshire St,Cambridge,MA, USA","(42.3696425, -71.09104734310242, 0.0)",42.369642,-71.091047,pink,2.0,17.0,purple,9,17.0
4,243,Broadway,#610,Cambridge,MA,1,"243 Broadway,Cambridge,MA, USA","(42.367239, -71.09552, 0.0)",42.367239,-71.095520,darkgreen,1.0,14.0,darkgreen,-1,14.0


In [165]:
df_final.to_excel("delivery_table.xlsx",index=False)